In [10]:
import sys
import plotly.io as pio
import plotly.graph_objects as go
sys.path.append("../..")

from evals.spelling_by_grade import assess_model_on_words, create_few_shot_grade_spelling_prompt, prepare_grade_spelling_eval
from evals.plot_utils import basic_bar_graph, create_table, nested_bar_graph

pio.renderers.default = "notebook"

words_by_grade = prepare_grade_spelling_eval("../data/GradeSpellingEval.txt", '-')
words_by_grade[3][:10]

[('able', 'A-B-L-E'),
 ('above', 'A-B-O-V-E'),
 ('afraid', 'A-F-R-A-I-D'),
 ('afternoon', 'A-F-T-E-R-N-O-O-N'),
 ('again', 'A-G-A-I-N'),
 ('age', 'A-G-E'),
 ('air', 'A-I-R'),
 ('airplane', 'A-I-R-P-L-A-N-E'),
 ('almost', 'A-L-M-O-S-T'),
 ('alone', 'A-L-O-N-E')]

In [ ]:
create_few_shot_grade_spelling_prompt('able', words_by_grade[3], 2)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformer_lens import HookedTransformer

SUPPORTED_MODELS = ['TransformerLens', 'HuggingFace']
MODEL_TYPE = 'HuggingFace' # TransformerLens or HuggingFace
assert MODEL_TYPE in SUPPORTED_MODELS

if MODEL_TYPE == 'HuggingFace':
    tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
    model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")
elif MODEL_TYPE == 'TransformerLens':
    model = HookedTransformer.from_pretrained("gpt-j-6b")
    tokenizer = model.tokenizer

In [ ]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)
torch.set_grad_enabled(False)
print(device, MODEL_TYPE)

prompts = ["Once upon a time", "Once upon a time"]
if MODEL_TYPE == 'HuggingFace':
    model.config.pad_token_id = tokenizer.eos_token_id # Prevent lots of info messages telling us it's doing this every prompt.
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = 'left'
    input_ids = tokenizer(prompts, padding=True, truncation=True, return_tensors="pt").to(model.device)
    outputs = model.generate(**input_ids, max_length=50, num_return_sequences=1, temperature=0.7, do_sample=True)
    response = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
elif MODEL_TYPE == 'TransformerLens':
    print(model.to_tokens(prompts[0]).shape[-1])
    response = [model.generate(prompt, max_new_tokens=10, temperature=0.7) for prompt in prompts]
print(response)

In [ ]:
shots = [0, 1, 2, 5, 10]
shot_data = {shot: {} for shot in shots}
for shot in shots:
    print(f"Checking {shot} {'shot' if shot == 1 else 'shots'}") # Was the grammar necessary? No, but it bugged me.
    shot_data[shot] = assess_model_on_words(model, MODEL_TYPE, tokenizer, words_by_grade, shot, 10)

In [ ]:
# Check how closely model's spelling ability matches grade.
shot_successes = {}
for shot in shots:
    successes = {grade: 0 for grade in words_by_grade.keys()}
    for grade in shot_data[shot].keys():
        successes[grade] = [word['answer'] in word['response'].upper() for word in shot_data[shot][grade]].count(True) / len(shot_data[shot][grade])
        successes[grade] = round(successes[grade], 3)
    shot_successes[shot] = successes
print(shot_successes)

In [ ]:
basic_bar_graph(shot_successes[2])

In [ ]:
# Check how closely model's spelling ability matches length of the word.
lengths = {shot: {i: {'success': 0, 'total': 0} for i in range(1, 13)} for shot in [0, 1, 2, 5, 10]}

for shot in lengths.keys():
    for grade in shot_data[shot].keys():
        for word in shot_data[shot][grade]:
            length = len(word['answer']) - word['answer'].count('-')
            lengths[shot][length]['total'] += 1
            if word['response'] in word['answer']:
                lengths[shot][length]['success'] += 1

# This is kinda janky, and I don't want to move it into plot_utils right now since it's very reliant on a particular data structure.
def success_rate_bar_graph(data):
    success_rates = {key: val['success'] / val['total'] if val['total'] > 0 else 0 for key, val in data.items()}
    sample_sizes = {key: val['total'] for key, val in data.items()}

    fig = go.Figure(go.Bar(
        x=list(success_rates.keys()), 
        y=list(success_rates.values()),
        text=[f"n={n}" for n in sample_sizes.values()],  # Sample sizes as text
        textposition='auto'  # Positioning the text inside the bars
    ))
    
    fig.update_layout(yaxis=dict(range=[0, 1]))
    fig.show()

success_rate_bar_graph(lengths[2])

In [ ]:
import numpy as np

green_shades = ['#a1d99b', '#74c476', '#31a354', '#006d2c', '#024736']
nested_bar_graph(shot_successes, green_shades)

In [ ]:
# Check to see how often the model gets the first letter right.
first_successes = {shot: {} for shot in shots}
for shot in shots:
    successes = {grade: 0 for grade in words_by_grade.keys()}
    for grade in range(1, 6):
        success_list = [word['response'].strip().upper().startswith(word['answer'][0]) for word in shot_data[shot][grade] if min(len(word['response']), len(word['answer'])) > 0]
        successes[grade] = round((success_list.count(True) / len(shot_data[shot][grade])), 3)                                                                                                                           
    first_successes[shot] = successes
print(first_successes)
nested_bar_graph(first_successes)

In [ ]:

table = create_table(shot_data[2])
table